# Lemon のモジュールを利用して、BERT-miniモデルを作成する

BERT mini model は 以下の設定
* epoch 5 までのモデル
* batch size 32,
* linearly decreasing learning rate from 3 ^ 10−5 with 50 warmup steps
* 16-bit precision optimization
* 1, 3, 5, 10, or 20 epochs depending on the dataset size
* The final model is the one from the epoch with the highest F1 score on the validation dataset.

In [1]:
dataset_out_root_dir = "../../data/lemon/datasets"
model_out_root_dir = "../../data/lemon/model/bert-mini"
dataset_names = [
    "structured_amazon_google",
    "structured_beer",
    "structured_dblp_acm",
    "structured_dblp_google_scholar",
    "structured_fodors_zagat",
    "structured_walmart_amazon",
    "structured_itunes_amazon",
    "dirty_dblp_acm",
    "dirty_dblp_google_scholar",
    "dirty_walmart_amazon",
    "dirty_itunes_amazon",
    "textual_abt_buy",
    "textual_company",
]
gpu_id = 5


In [2]:
# torchモジュールの読み込み前に、利用できるGPUを指定しておく
## これをやらないと、システム内のＧＰＵすべてを利用してしまう
import os

os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_id}"


In [3]:
import torch

print("CUDA =", torch.cuda.is_available())
print("CUDA DEVICES =", torch.cuda.device_count())
print("CUDA CURRENT DEVICE_ID = ", torch.cuda.current_device())


CUDA = True
CUDA DEVICES = 1
CUDA CURRENT DEVICE_ID =  0


In [4]:
# Set Random Seeds and Reproducibility
import random

import numpy as np


def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch``
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


set_seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


In [5]:
import pickle
import pathlib

import lemon.utils.datasets.deepmatcher
from transformers import AutoModelForSequenceClassification
from transformers.trainer_callback import TrainerState


def get_best_model_checkpoint_dir(checkpoints_dir_path: pathlib.Path):
    last_checkpoints_dir = sorted(
        checkpoints_dir_path.iterdir(), key=lambda x: int(str(x).split("-")[-1])
    )[-1]
    state = TrainerState.load_from_json(
        f"{str(last_checkpoints_dir)}/trainer_state.json"
    )
    return pathlib.Path((state.best_model_checkpoint))


for dataset_name in dataset_names:
    print("=============================")
    print(dataset_name)
    print("=============================")
    load_dataset_func = getattr(lemon.utils.datasets.deepmatcher, dataset_name)
    dataset = load_dataset_func(dataset_out_root_dir)
    output_dir_path = (
        pathlib.Path(model_out_root_dir) / dataset_name
    )
    output_dir_path.mkdir(parents=True, exist_ok=True)

    matcher = lemon.utils.matchers.TransformerMatcher(
        "google/bert_uncased_L-4_H-256_A-4",
        tokenizer_args={"model_max_length": 256},
        training_args={
            "output_dir": str(output_dir_path / "checkpoints"),
            "logging_dir": str(output_dir_path / "logs"),
            "per_device_train_batch_size": 32,
            "learning_rate": 3e-5,
            "warmup_steps": 50,
            "fp16": True,
            "num_train_epochs": 20,
        },
    )
    print("training...")
    ret = matcher.fit(
        dataset.train.records.a,
        dataset.train.records.b,
        dataset.train.record_id_pairs,
        dataset.train.labels,
        dataset.val.record_id_pairs,
        dataset.val.labels,
    )
    display(ret)
    print("training...done")
    eval_result = matcher.evaluate(
        dataset.test.records.a,
        dataset.test.records.b,
        dataset.test.record_id_pairs,
        dataset.test.labels,
    )
    print(eval_result)
    with (output_dir_path / "eval_result.pickle").open(
        "wb"
    ) as f:
        pickle.dump(eval_result, f)
    print("reload test...")
    print(get_best_model_checkpoint_dir(output_dir_path / "checkpoints"))
    bert_mini_model_reload = AutoModelForSequenceClassification.from_pretrained(
        get_best_model_checkpoint_dir(output_dir_path / "checkpoints")
    )
    matcher_reload = lemon.utils.matchers.TransformerMatcher(bert_mini_model_reload)
    eval_result_reload = matcher_reload.evaluate(
        dataset.test.records.a,
        dataset.test.records.b,
        dataset.test.record_id_pairs,
        dataset.test.labels,
    )
    print(eval_result_reload)
    assert eval_result == eval_result_reload


structured_amazon_google


Some weights of the model checkpoint at google/bert_uncased_L-4_H-256_A-4 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

training...


Using amp fp16 backend
***** Running training *****
  Num examples = 6874
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4300


Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,0.217900,0.205045,0.529563,0.664516,0.440171
2,0.151300,0.176664,0.630021,0.623431,0.636752
3,0.159300,0.165656,0.645570,0.637500,0.653846
4,0.220100,0.175556,0.685225,0.686695,0.683761
5,0.156600,0.185241,0.688797,0.669355,0.709402
6,0.159700,0.179734,0.710680,0.651246,0.782051
7,0.078000,0.177274,0.696356,0.661538,0.735043
8,0.126000,0.196142,0.695297,0.666667,0.726496
9,0.088500,0.205477,0.694184,0.618729,0.790598
10,0.130700,0.229534,0.695327,0.617940,0.794872


...

PyTorch: setting up devices
Using amp fp16 backend
***** Running Prediction *****
  Num examples = 22503
  Batch size = 64


loading configuration file ../../data/lemon/model/bert-mini/textual_company/checkpoints/checkpoint-10565/config.json
Model config BertConfig {
  "_name_or_path": "google/bert_uncased_L-4_H-256_A-4",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 4,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ../../data/lemon/model/bert-mini/textual_company/checkpoints/checkpoint-10565/pytorch_model.bin
All model checkpoint weights we

{'precision': 0.8327693677649154, 'recall': 0.8290780141843972, 'f1': 0.8309195912927589}
reload test...
../../data/lemon/model/bert-mini/textual_company/checkpoints/checkpoint-10565


PyTorch: setting up devices
Using amp fp16 backend
***** Running Prediction *****
  Num examples = 22503
  Batch size = 64


{'precision': 0.8327693677649154, 'recall': 0.8290780141843972, 'f1': 0.8309195912927589}
